## Downloading Videos from sheet to my google drive

In [ ]:
import os
import pandas as pd
import requests

Step 1: Load Excel file
excel_path = "/content/Assignment Data.xlsx"  # Replace with your file path
data = pd.read_excel(excel_path)

# Step 2: Create a directory to store the videos
dataset_folder = "/content/drive/MyDrive/assi_Data_video/video_dataset"
os.makedirs(dataset_folder, exist_ok=True)

# Step 3: Prepare a list for metadata
metadata = []

# Step 4: Loop through the data and download videos
for index, row in data.iterrows():
    score = row['Performance']  # Adjust column name to match Excel
    video_url = row['Video URL']  # Adjust column name to match Excel

    try:
        # Download video using requests
        video_filename = f"{index}_video.mp4"
        video_path = os.path.join(dataset_folder, video_filename)

        # Stream and save video
        with requests.get(video_url, stream=True) as response:
            response.raise_for_status()  # Raise an error for bad status codes
            with open(video_path, "wb") as video_file:
                for chunk in response.iter_content(chunk_size=8192):  # Download in chunks
                    video_file.write(chunk)

        # Append to metadata
        metadata.append({"video_path": video_path, "score": score})
        print(f"Downloaded: {video_filename}")

    except Exception as e:
        print(f"Error downloading video from {video_url}: {e}")

# Step 5: Save metadata to a CSV file
metadata_file = os.path.join(dataset_folder, "metadata.csv")
pd.DataFrame(metadata).to_csv(metadata_file, index=False)
print(f"Metadata saved at: {metadata_file}")


## Creating a metadata CSV file containing video names and scores

In [ ]:
import pandas as pd

# Load the existing metadata file
metadata_path = "/content/drive/MyDrive/assi_Data_video/video_dataset/metadata.csv"
metadata = pd.read_csv(metadata_path)

# Extract only the file names from the full paths
metadata['video_name'] = metadata['video_path'].apply(lambda x: os.path.basename(x))

# Remove the old 'video_path' column and reorder columns
metadata = metadata[['video_name', 'score']]

# Save the updated metadata
updated_metadata_path = "/content/drive/MyDrive/assi_Data_video/video_dataset/updated_metadata.csv"
metadata.to_csv(updated_metadata_path, index=False)
print(f"Updated metadata saved at: {updated_metadata_path}")


Updated metadata saved at: /content/drive/MyDrive/assi_Data_video/video_dataset/updated_metadata.csv


In [ ]:
pip install face_recognition opencv-python-headless pandas deepface moviepy imagehash scikit-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 9.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 82.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 110.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.7 MB/s eta 0:00:00
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566162 sha256=a6d9183b87ed021a5f657a7e8fe813dd15115fb46fe9748576b13591590d49d7
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cf

### Importing Required Libraries

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from moviepy.editor import VideoFileClip
import face_recognition
from sklearn.cluster import DBSCAN
from collections import defaultdict
from PIL import Image
import imagehash


  if event.key is 'enter':



## Removing Duplicate Videos from Dataset using video hash

In [ ]:
def extract_video_hash(video_path):
    """Generate a perceptual hash for a video based on its frames."""
    video = cv2.VideoCapture(video_path)
    frame_hashes = []
    frame_count = 0

    while True:
        ret, frame = video.read()
        if not ret:
            break
        frame_count += 1

        # Sample one frame every 30 frames
        if frame_count % 30 == 0:
            frame = cv2.resize(frame, (200, 200))  # Resize for uniform hashing
            frame_hashes.append(str(imagehash.phash(Image.fromarray(frame))))

    video.release()
    return "".join(frame_hashes)

def remove_duplicates(video_paths):
    """Remove duplicate videos based on hash comparison."""
    seen_hashes = set()
    unique_videos = []

    for video_path in video_paths:
        video_hash = extract_video_hash(video_path)
        if video_hash not in seen_hashes:
            seen_hashes.add(video_hash)
            unique_videos.append(video_path)

    return unique_videos


In [ ]:
metadata_file = "/content/drive/MyDrive/assi_Data_video/video_dataset/metadata.csv"
metadata = pd.read_csv(metadata_file)

video_paths = metadata['video_path'].tolist()
scores = metadata['score'].tolist()

# Step 1: Remove duplicates
unique_videos = remove_duplicates(video_paths)

# Step 2: Extract faces and recognize unique influencers
influencer_data = []
all_encodings = []



## Saving unique videos to my google drive

In [ ]:
import os
import shutil

# Path to the folder where unique videos will be saved
unique_videos_folder = "/content/drive/MyDrive/assi_Data_video/unique_videos"
os.makedirs(unique_videos_folder, exist_ok=True)  # Create the folder if it doesn't exist

# Copy each unique video to the new folder
for video_path in unique_videos:
    video_name = os.path.basename(video_path)  # Extract the video file name
    destination_path = os.path.join(unique_videos_folder, video_name)  # Define the destination path
    shutil.copy(video_path, destination_path)  # Copy the video file
    print(f"Copied {video_name} to {unique_videos_folder}")

print("All unique videos have been saved to the new folder.")

Copied 0_video.mp4 to /content/drive/MyDrive/assi_Data_video/unique_videos
Copied 1_video.mp4 to /content/drive/MyDrive/assi_Data_video/unique_videos
Copied 2_video.mp4 to /content/drive/MyDrive/assi_Data_video/unique_videos
Copied 3_video.mp4 to /content/drive/MyDrive/assi_Data_video/unique_videos
Copied 5_video.mp4 to /content/drive/MyDrive/assi_Data_video/unique_videos
Copied 6_video.mp4 to /content/drive/MyDrive/assi_Data_video/unique_videos
Copied 7_video.mp4 to /content/drive/MyDrive/assi_Data_video/unique_videos
Copied 11_video.mp4 to /content/drive/MyDrive/assi_Data_video/unique_videos
Copied 12_video.mp4 to /content/drive/MyDrive/assi_Data_video/unique_videos
Copied 14_video.mp4 to /content/drive/MyDrive/assi_Data_video/unique_videos
Copied 15_video.mp4 to /content/drive/MyDrive/assi_Data_video/unique_videos
Copied 18_video.mp4 to /content/drive/MyDrive/assi_Data_video/unique_videos
Copied 23_video.mp4 to /content/drive/MyDrive/assi_Data_video/unique_videos
Copied 24_video.mp4